In [1]:
from estnltk.text import Text

In [2]:
t = Text('3 ust 5ndal korrusel.').tag_layer()

In [3]:
t.morf_analysis

SL[SL[Span(3, {'clitic': '', 'partofspeech': 'N', 'root': '3', 'lemma': '3', 'form': '?', 'ending': '0', 'root_tokens': ['3']})],
SL[Span(ust, {'clitic': '', 'partofspeech': 'S', 'root': 'uks', 'lemma': 'uks', 'form': 'sg p', 'ending': 't', 'root_tokens': ['uks']})],
SL[Span(5ndal, {'clitic': '', 'partofspeech': 'O', 'root': '5', 'lemma': '5', 'form': 'sg ad', 'ending': 'l', 'root_tokens': ['5']})],
SL[Span(korrusel, {'clitic': '', 'partofspeech': 'S', 'root': 'korrus', 'lemma': 'korrus', 'form': 'sg ad', 'ending': 'l', 'root_tokens': ['korrus']})],
SL[Span(., {'clitic': '', 'partofspeech': 'Z', 'root': '.', 'lemma': '.', 'form': '', 'ending': '', 'root_tokens': ['.']})]]